# This script was used to see how electrodes correlate across subjects. See another from: https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1006769
#


In [1]:
import numpy as np

from gesture.utils import get_epoch,get_good_channels
import matplotlib.pyplot as plt
import scipy.io
from gesture.config import *
import h5py
import pandas as pd
from gesture.utils import sub_sf_dict

2024-08-03 09:47:43 - Start Program

pre_all: Running from CMD.


In [2]:
filename=meta_dir+'ele_anat_position.npy'
ele = np.load(filename,allow_pickle=True).item()
good_channels=get_good_channels()

In [3]:
sid=2
info_dict=sub_sf_dict()
sf=info_dict[str(sid)]

tmin=0
tmax=0.5
epochs=get_epoch(sid, sf,scaler='no',trigger='EMG',tmin=tmin,tmax=tmax)
#epoch1=epochs['1']# 20 trials. 8001 time points per trial for 8s.
#epoch2=epochs['2']
epoch3=epochs['3'] # simple grasp
total_ch_number=len(epoch3.info.ch_names)
#epoch4=epochs['4']

In [4]:
# mark bad channels
good_channel=good_channels['sid'+str(sid)] # start from 1 (Matlab)
good_channel=[i-1 for i in good_channel]
bad_channels_ind=[i for i in range(len(epoch3.ch_names)-7) if i not in good_channel]
bad_channels=[epoch3.ch_names[c] for c in bad_channels_ind]
epoch3.load_data()
epoch3.drop_channels(bad_channels)


Number of events,20
Events,3: 20
Time range,0.000 – 0.500 sec
Baseline,off


### For sid2 with total 128 channels = 115good_channels (should be same length as new in below)
### +6bad_channels +2*emg+emg_diff+emg_diff_env+env_smoth_deriv+trigger_index+trigger_index_emg

In [ ]:
# populate new channel name
info_tmp=epoch3.info
old=info_tmp.ch_names
new=ele['sid'+str(sid)]['ana_label_id']
mapping={old[i]:new[i] for i in range(len(new))}
mne.rename_channels(info_tmp,mapping)
epoch3.info=info_tmp

assert len(good_channel)==len(new)
assert total_ch_number==len(new)+len(bad_channels)+7

In [ ]:
ch_names=epoch3.info.ch_names 

In [ ]:
sorted_index=sorted(range(len(ch_names)),key=ch_names.__getitem__)
ordered_ch_names=[ch_names[c] for c in sorted_index]

In [ ]:
epoch3_ordered=epoch3.copy().reorder_channels(ordered_ch_names)

In [ ]:
epoch3_ordered.plot()

### re-ordered CC:

In [ ]:
epoch=epoch3_ordered.copy().load_data().filter(l_freq=0.1,h_freq=3,picks=['eeg'])
matrixes=[]
data=epoch.get_data(picks=['eeg'])
ch_number=data.shape[1]
for trial in data: # trial: (115, 1000)
    dataframe=pd.DataFrame(data=trial.transpose(),columns=[str(i) for i in range(ch_number)])
    matrix = dataframe.corr(method='pearson')
    matrixes.append(matrix)
matrix_avg=np.average(np.asarray(matrixes),axis=0)

In [ ]:
%matplotlib inline
#%matplotlib qt
#%gui qt
plt.imshow(matrix_avg, cmap='Blues')
plt.colorbar()
variables=ordered_ch_names[:-7] #ele['sid2']['ana_label_id']
plt.xticks(range(len(matrix)), variables, rotation=90, ha='right')
plt.yticks(range(len(matrix)), variables)
plt.show()

### Wrap up above in a function, calculate_CC, and calculate the cross-subject CC between different subjects

In [1]:
from gesture.group_analysis.util import calculate_CC

sid1=2
sid2=4
sf1=1000
sf2=1000

matrixes,ordered_ch_names=calculate_CC(sid1, sf1, sid2, sf2, f1=0.1, f2=3,trigger='eeg',re_ordered=True)

2024-08-03 10:57:40 - Start Program

pre_all: Running from CMD.


In [2]:
matrixes.shape

(0,)

In [ ]:
%matplotlib inline
#%matplotlib qt
fig,ax=plt.subplots()
cc_avg=np.average(matrixes,axis=0)
im=ax.imshow(cc_avg)
fig.colorbar(im)
plt.yticks(range(len(ordered_ch_names[0])), ordered_ch_names[0], rotation=0, ha='right')
plt.xticks(range(len(ordered_ch_names[1])), ordered_ch_names[1],rotation=90)
plt.show()

![](img/6.png)

CC could be 0.4 even if shift sid 2 and 4 at random period.

Below is cc obtained by aligning using EEG trigger:  matrixes,ordered_ch_names=calculate_CC(sid1, sf1, sid2, sf2, f1=0.1, f2=3,trigger='eeg',re_ordered=True):

![](img/7.png)

is it possible? still need investigation.